In [1]:
import multiprocessing
multiprocessing.cpu_count()

8

## Multiprocessing

In [3]:
import os
from multiprocessing import Process

def run_proc(name):   #子进程要执行的代码
    print("Run child process %s (%s)..." % (name, os.getpid()))

if __name__=='__main__':
    print("Parent process %s." % os.getpid())
    for i in range(5):
        p = Process(target = run_proc, args = (str(i), ))
        print("Child process will start")
        p.start()
    p.join()
    print("Child process end.")

Parent process 21454.
Child process will start
Child process will start
Child process will start
Run child process 0 (21468)...
Run child process 1 (21469)...
Child process will start
Run child process 2 (21470)...
Child process will start
Run child process 3 (21471)...
Run child process 4 (21472)...
Child process end.


In [ ]:
def run_proc(name):
    print('Run child process %s (%s)...' % (name, os.getpid()))

if __name__=='__main__':
    print('Parent process %s.' % os.getpid())
    p = Process(target=run_proc, args=('test',))
    print('Child process will start.')
    p.start()
    p.join()
    print('Child process end.')

In [ ]:
from multiprocessing import Pool
import os, time, random

def long_time_task(name):
    print("Run task %s (%s)..." % (name, os.getpid()))
    start = time.time()
    time.sleep(random.random() * 3)
    end = time.time()
    print('Task %s runs %0.2f seconds.' % (name, (end - start)))
    
if __name__=='__main__':
    print("Parent process %s." % os.getpid())
    p = Pool(4)
    for i in range(5):
        p.apply_async(long_time_task, args = (i, ))
    print("Waiting for all subprocesses done...")
    p.close()
    p.join()
    print("All subprocesses done.")

In [7]:
import subprocess

print('$ nslookup www.python.org')
r = subprocess.call(['nslookup', 'www.python.org'])
print('Exit code:', r)

$ nslookup www.python.org
Exit code: 0


In [10]:
from multiprocessing import Process, Queue
import os, time, random

# 写数据进程执行的代码:
def write(q):
    print('Process to write: %s' % os.getpid())
    for value in ['A', 'B', 'C']:
        print('Put %s to queue...' % value)
        q.put(value)
        time.sleep(random.random())

# 读数据进程执行的代码:
def read(q):
    print('Process to read: %s' % os.getpid())
    while True:
        value = q.get(True)
        print('Get %s from queue.' % value)

if __name__=='__main__':
    # 父进程创建Queue，并传给各个子进程：
    q = Queue()
    pw = Process(target=write, args=(q,))
    pr = Process(target=read, args=(q,))
    # 启动子进程pw，写入:
    pw.start()
    # 启动子进程pr，读取:
    pr.start()
    # 等待pw结束:
    pw.join()
    # pr进程里是死循环，无法等待其结束，只能强行终止:
    pr.terminate()

## Fork

In [4]:
import os
print("Process (%s) start..." % os.getpid())    #获取当前进程的id
pid = os.fork()
if pid == 0:
    print("I am child process (%s) and my parent is %s" % (os.getpid(), os.getppid()))
else:
    print('I (%s) just created a child process (%s).' % (os.getpid(), pid))

Process (21454) start...
I (21454) just created a child process (21474).
I am child process (21474) and my parent is 21454


## Pool

In [5]:
from multiprocessing import Pool
import os, time, random

def long_time_task(name):
    print("Run task %s (%s)..." % (name, os.getpid()))
    start = time.time()
    time.sleep(random.random() * 3)
    end = time.time()
    print('Task %s runs %0.2f seconds.' % (name, (end - start)))
    
if __name__=='__main__':
    print("Parent process %s." % os.getpid())
    p = Pool(4)
    for i in range(5):
        p.apply_async(long_time_task, args = (i, ))
    print("Waiting for all subprocesses done...")
    p.close()
    p.join()
    print("All subprocesses done.")

Parent process 21454.
Run task 1 (21476)...
Run task 2 (21477)...
Run task 0 (21475)...
Run task 3 (21478)...
Task 0 runs 0.10 seconds.
Run task 4 (21475)...
Waiting for all subprocesses done...
Task 2 runs 0.34 seconds.
Task 4 runs 0.56 seconds.
Task 3 runs 0.82 seconds.
Task 1 runs 2.58 seconds.
All subprocesses done.


## Subprocess

In [6]:
import subprocess

print("$ nslookup www.python.org")
r = subprocess.call(['nslookup', 'www.python.org'])
print("Exit code:", r)

$ nslookup www.python.org
Exit code: 0
